In [1]:
import pandas as pd
df = pd.read_csv(encoding='MacRoman',
    filepath_or_buffer='/kaggle/input/airplane-crashes-and-fatalities-upto-2023/Airplane_Crashes_and_Fatalities_Since_1908_t0_2023.csv',
                parse_dates=['Date'])
df['year'] = df['Date'].dt.year
df.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary,year
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,1.0,1.0,1.0,0.0,0.0,"During a demonstration flight, a U.S. Army fly...",1908
1,1909-09-07,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...,1909
2,1912-07-12,6:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,1912
3,1913-08-06,NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1.0,0.0,1.0,1.0,0.0,1.0,0.0,The first fatal airplane accident in Canada oc...,1913
4,1913-09-09,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,NaN,NaN,14.0,NaN,NaN,0.0,The airship flew into a thunderstorm and encou...,1913


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   4998 non-null   datetime64[ns]
 1   Time                   3486 non-null   object        
 2   Location               4994 non-null   object        
 3   Operator               4988 non-null   object        
 4   Flight #               1329 non-null   object        
 5   Route                  4221 non-null   object        
 6   AC Type                4983 non-null   object        
 7   Registration           4724 non-null   object        
 8   cn/ln                  4330 non-null   object        
 9   Aboard                 4980 non-null   float64       
 10  Aboard Passangers      4769 non-null   float64       
 11  Aboard Crew            4772 non-null   float64       
 12  Fatalities             4990 non-null   float64       
 13  Fat

In [3]:
df.nunique()

Date                     4570
Time                     1060
Location                 4122
Operator                 2264
Flight #                  880
Route                    3825
AC Type                  2463
Registration             4689
cn/ln                    3818
Aboard                    244
Aboard Passangers         234
Aboard Crew                31
Fatalities                199
Fatalities Passangers     190
Fatalities Crew            28
Ground                     51
Summary                  4839
year                      113
dtype: int64

In [4]:
from plotly.express import histogram
for column in ['year', 'Aboard', 'Aboard Passangers', 'Aboard Crew', 'Fatalities', 'Fatalities Passangers',
              'Fatalities Crew', 'Ground']:
    histogram(data_frame=df, x=column, log_y=column != 'year').show()

In [5]:
for column in ['Operator', 'AC Type']:
    histogram(data_frame=df[column].value_counts().to_frame().reset_index().head(n=50), x=column, y='count').show()

In [6]:
from plotly.express import scatter
scatter(data_frame=df, x='Aboard', y='Fatalities', color='year', log_x=True, log_y=True)

In [7]:
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from plotly.express import bar

TOKENIZER = RegexpTokenizer(r'\w+')

def find_term(arg:str, terms: list):
    lower = TOKENIZER.tokenize(text=arg.lower())
    for term in terms:
        if term in lower:
            return term
    return 'none'

DISCARD = {}
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words='english', max_df=1.0, min_df=0.01)
fit_result = vectorizer.fit_transform(raw_documents=df.dropna(subset='Summary')['Summary'].values)
word_weight_df = pd.DataFrame(data={'word': vectorizer.get_feature_names_out(), 'weight': fit_result.toarray().sum(axis=0),}).sort_values(ascending=False, by='weight')
bar(data_frame=word_weight_df.head(n=30), x='word', y='weight')